In [1]:
import numpy as np
import pandas as pd
from scipy.spatial import Delaunay
import tkinter as tk
from tkinter import filedialog

ModuleNotFoundError: No module named 'numpy'

In [ ]:
# ----------------------------------------------------------
# GUI file selector
# ----------------------------------------------------------
root = tk.Tk()
root.withdraw()  # Hide the small tk window

file_path = filedialog.askopenfilename(
    title="Select the outletSliceData CSV file",
    filetypes=[("CSV Files", "*.csv"), ("All Files", "*.*")]
)

if not file_path:
    raise RuntimeError("No file selected.")

print("Selected file:", file_path)


In [ ]:
# ----------------------------------------------------------
# Load data
# ----------------------------------------------------------
df = pd.read_csv(file_path)

# Update column names as needed
y = df["Point1"].values
z = df["Point2"].values
M = df["Mach"].values

# All x = constant; triangulate in y–z plane
points_2d = np.column_stack((y, z))

print(points_2d)

In [ ]:
# ----------------------------------------------------------
# Delaunay triangulation
# ----------------------------------------------------------
tri = Delaunay(points_2d)
triangles = tri.simplices  # (Ntri, 3)

# ----------------------------------------------------------
# Compute triangle areas
# ----------------------------------------------------------
A = np.zeros(len(triangles))

for i, (a, b, c) in enumerate(triangles):
    y1, z1 = points_2d[a]
    y2, z2 = points_2d[b]
    y3, z3 = points_2d[c]

    area = 0.5 * abs((y2 - y1) * (z3 - z1) - (y3 - y1) * (z2 - z1))
    A[i] = area

# ----------------------------------------------------------
# Average Mach on each triangle
# ----------------------------------------------------------
M_tri = M[triangles].mean(axis=1)

# ----------------------------------------------------------
# Area-weighted average Mach
# ----------------------------------------------------------
area_avg_Mach = np.sum(M_tri * A) / np.sum(A)

print("\nArea-averaged Mach number:", area_avg_Mach)
